# GMM4PR Analysis - Step 1: Load GMM
This notebook loads trained GMM models from fit_gmm2.py

In [2]:
import os
import torch
from config import get_config
from utils import GMM4PR, get_dataset, build_model, build_decoder_from_flag, eval_acc

# Specify which trained GMM to load
CONFIG_NAME = "test_1"  # Must match the config name used in fit_gmm2.py
CHECKPOINT_DIR = "./ckp/gmm_ckp/"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load configuration
cfg = get_config(CONFIG_NAME)
checkpoint_path = os.path.join(CHECKPOINT_DIR, f"gmm_{cfg.exp_name}.pt")

# Load dataset to get out_shape
dataset, num_classes, out_shape = get_dataset(cfg.dataset, cfg.data_root, train=True, resize=True)
print(f"Dataset: {cfg.dataset}, Classes: {num_classes}, Shape: {out_shape}")

# load classifier weights 
print(f"Using device: {DEVICE}")
print(f"\nLoading classifier: {cfg.arch}")
model, feat_extractor = build_model(cfg.arch, num_classes, DEVICE)

if not os.path.isfile(cfg.clf_ckpt):
    raise FileNotFoundError(f"Classifier not found: {cfg.clf_ckpt}")

state = torch.load(cfg.clf_ckpt, map_location="cpu")
state = state.get("state_dict", state.get("model_state", state))
state = {k.replace("module.", ""): v for k, v in state.items()}

model.load_state_dict(state, strict=False)
model = model.to(DEVICE).eval()
for p in model.parameters():
    p.requires_grad = False

feat_extractor = feat_extractor.to(DEVICE).eval()
for p in feat_extractor.parameters():
    p.requires_grad = False

# Check parameter sharing
model_params = {id(p) for p in model.parameters()}
feat_params  = {id(p) for p in feat_extractor.parameters()}
shared = model_params & feat_params

print(f"[check] model params: {len(model_params)}, feat_extractor params: {len(feat_params)}")
if shared:
    print(f"[check] They share {len(shared)} parameters.")
else:
    print("[check] No shared parameters.")

print("Evaluating clean accuracy...")
eval_acc(model, dataset, DEVICE) # accuracy on training set

# Build decoder/up_sampler (if used during training)
up_sampler = None
if cfg.use_decoder:
    print(f"\nBuilding decoder: {cfg.decoder_backend}")
    up_sampler = build_decoder_from_flag(
        cfg.decoder_backend,
        cfg.latent_dim,
        out_shape,
        DEVICE
    )
    print(f"✓ Decoder {cfg.decoder_backend} built successfully!")
else:
    print(f"\nNo decoder used (use_decoder={cfg.use_decoder})")

# Load the trained GMM model
print(f"Loading from: {checkpoint_path}")
gmm = GMM4PR.load_from_checkpoint(
    checkpoint_path,
    feat_extractor=feat_extractor, 
    up_sampler=up_sampler,      
    map_location=DEVICE,
    strict=True
)

gmm = gmm.to(DEVICE).eval()


Files already downloaded and verified
Dataset: cifar10, Classes: 10, Shape: (3, 224, 224)
Using device: cuda

Loading classifier: resnet18
[check] model params: 62, feat_extractor params: 60
[check] They share 60 parameters.
Evaluating clean accuracy...


/tmp/ipykernel_2556197/2110917319.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(cfg.clf_ckpt, map_location="cpu")


[clf] accuracy=95.76%

Building decoder: bicubic_trainable
[Decoder 'bicubic'] 37,779 params
✓ Decoder bicubic_trainable built successfully!
Loading from: ./ckp/gmm_ckp/gmm_test_1.pt


/home/wang3_z@WMGDS.WMG.WARWICK.AC.UK/WSPR/utils/gmm4pr.py:985: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(path, map_location=map_location)


[Params] Shared trunk: 263,680 | pi: 3,591 | mu: 919,296, | cov: 60,017,920 | Total: 60,940,807
Model loaded from ./ckp/gmm_ckp/gmm_test_1.pt


## 2. Compute Probabilistic Robustness (PR)

In [5]:
from utils import compute_pr_on_clean_correct

# Create DataLoader for evaluation
loader = torch.utils.data.DataLoader(
    dataset, 
    batch_size=cfg.batch_size,
    shuffle=False,
    num_workers=cfg.num_workers,
    pin_memory=True
)

# Set number of samples per image
S_SAMPLES = 100  # Number of Monte Carlo samples per image

# Optional: Specify batch indices to evaluate (None = all batches)
# BATCH_INDICES = None  # Evaluate all batches
# BATCH_INDICES = [0, 1, 2, 3, 4]  # Evaluate only first 5 batches
BATCH_INDICES = range(50)  # Evaluate batches 100 to 199

print(f"\nComputing PR on clean-correct samples...")
print(f"  GMM: {cfg.exp_name}")
print(f"  Samples per image: {S_SAMPLES}")
print(f"  Batch indices: {'All' if BATCH_INDICES is None else BATCH_INDICES}")

# Compute PR
pr, n_used, clean_acc = compute_pr_on_clean_correct(
    model=model,
    gmm=gmm,
    loader=loader,
    out_shape=out_shape,
    device=DEVICE,
    num_samples=500,
    batch_indices=range(10),
    temperature=1,  # Very low temperature should approximate hard sampling
    use_soft_sampling=False,
    chunk_size=32
)

print(f"\n{'='*60}")
print(f"Results:")
print(f"  Clean Accuracy: {clean_acc*100:.2f}%")
print(f"  Probabilistic Robustness (PR): {pr:.4f}")
print(f"  Clean-correct samples used: {n_used}")
print(f"{'='*60}")


Computing PR on clean-correct samples...
  GMM: test_1
  Samples per image: 100
  Batch indices: range(0, 50)
[PR@clean] used=309 / seen=320 (clean acc=96.56%), num_samples=500, chunk_size=32 → PR=0.9594 [method: hard (categorical)]

Results:
  Clean Accuracy: 96.56%
  Probabilistic Robustness (PR): 0.9594
  Clean-correct samples used: 309
